In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import cv2
import kagglehub

In [2]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
    self.pool1 = nn.MaxPool2d(2, 2)
    self.batchnorm1 = nn.BatchNorm2d(num_features=32)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(64 * 32 * 32, 128)
    self.out = nn.Linear(128, 4)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = self.batchnorm1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)
    x = self.flatten(x)
    x = F.relu(self.fc1(x))
    x = self.out(x)
    return x

In [3]:
# Download latest version
path = kagglehub.dataset_download("borhanitrash/alzheimer-mri-disease-classification-dataset")
disease_label_from_category = {
    0: "Mild Demented",
    1: "Moderate Demented",
    2: "Non Demented",
    3: "Very Mild Demented",
}

100%|██████████| 26.0M/26.0M [00:00<00:00, 92.4MB/s]

Extracting files...


In [4]:
train_df = pd.read_parquet(f"{path}/Alzheimer MRI Disease Classification Dataset/Data/train-00000-of-00001-c08a401c53fe5312.parquet", engine="pyarrow")

In [5]:
test_df = pd.read_parquet(f"{path}/Alzheimer MRI Disease Classification Dataset/Data/test-00000-of-00001-44110b9df98c5585.parquet", engine="pyarrow")

In [6]:
def dict_to_image(image_dict):
    if isinstance(image_dict, dict) and 'bytes' in image_dict:
        byte_string = image_dict['bytes']
        nparr = np.frombuffer(byte_string, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)
        return img
    else:
        raise TypeError(f"Expected dictionary with 'bytes' key, got {type(image_dict)}")

train_df['img_arr'] = train_df['image'].apply(dict_to_image)
train_df.drop("image", axis=1, inplace=True)

test_df['img_arr'] = test_df['image'].apply(dict_to_image)
test_df.drop("image", axis=1, inplace=True)

In [7]:
class MRIDataset(Dataset):
  def __init__(self, df):
    self.df = df

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    image = self.df.iloc[idx]['img_arr']
    label = self.df.iloc[idx]['label']

    image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)
    label = torch.tensor(label, dtype=torch.long)

    return image, label

train_dataset = MRIDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [8]:
def train_model(model, loader, optimizer, num_epochs):
  criterion = nn.CrossEntropyLoss()
  train_losses = []
  for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(loader, 0):
      inputs, labels = data[0].to(device), data[1].to(device)
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
    epoch_loss = running_loss / len(loader)
    train_losses.append(epoch_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
  print("Finished Training")
  return model, train_losses

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
model, train_losses = train_model(model, train_loader, optimizer, 8)

Epoch 1/8, Loss: 1.4818
Epoch 2/8, Loss: 0.5378
Epoch 3/8, Loss: 0.2427
Epoch 4/8, Loss: 0.0894
Epoch 5/8, Loss: 0.0293
Epoch 6/8, Loss: 0.0095
Epoch 7/8, Loss: 0.0071
Epoch 8/8, Loss: 0.0018
Finished Training


In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [11]:
def predict(model, loader, device):
  model.eval()
  predictions = []
  true_labels = []

  with torch.no_grad():
    for images, labels in loader:
      images = images.to(device)
      outputs = model(images)
      _, preds = torch.max(outputs, 1)
      predictions.extend(preds.cpu().numpy())
      true_labels.extend(labels.cpu().numpy())
  return predictions, true_labels

def result_summary(predictions, true_labels):
  accuracy = accuracy_score(true_labels, predictions)
  print(f"Accuracy: {accuracy:.4f}")
  conf_matrix = confusion_matrix(true_labels, predictions)
  print("Confusion Matrix:")
  print(conf_matrix)

In [12]:
predictions, true_labels = predict(model, train_loader, device)
result_summary(predictions, true_labels)

Accuracy: 1.0000
Confusion Matrix:
[[ 724    0    0    0]
 [   0   49    0    0]
 [   0    0 2566    0]
 [   0    0    0 1781]]


In [13]:
test_dataset = MRIDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
predictions_test, test_labels = predict(model, test_loader, device)
result_summary(predictions_test, test_labels)

Accuracy: 0.9680
Confusion Matrix:
[[158   0   7   7]
 [  0  14   0   1]
 [  0   0 631   3]
 [  0   0  23 436]]


In [15]:
torch.save(model, 'alzheimer_cnn.pth')

In [16]:
from google.colab import files
files.download('alzheimer_cnn.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>